In [1]:
import networkx as nx
from pyvis.network import Network
import random
import os

In [7]:

def load_random_graph(directory):
    files = [file for file in os.listdir(directory) if file.endswith('.graphml')] # or other format
    random_file = random.choice(files)
    path = os.path.join(directory, random_file)
    print(f'Loading graph from {path}')
    return nx.read_graphml(path)  # Change this according to your graph format

def get_root_nodes(G):
    return [n for n, d in G.in_degree() if d == 0]

def plot_graph_with_sccs(G, sccs):
    net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', notebook=True)

    # Add nodes and edges
    for node in G.nodes:
        net.add_node(node, label=str(node), title=str(node))

    for edge in G.edges:
        net.add_edge(edge[0], edge[1])

    #loop over all sccs, select a unique color for each scc and color the nodes in the scc
    colors = ['#'+str(hex(random.randint(0, 16777215)))[2:] for i in range(len(sccs))]
    for i, scc in enumerate(sccs):
        for node in scc:
            net.get_node(node)['color'] = colors[i]
            

    # Generate network with specific layout
    net.from_nx(G)
    net.show('graph_viz_sccs.html')

def plot_graph_with_roots(G, roots):
    net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', notebook=True)

    # Add nodes and edges
    for node in G.nodes:
        net.add_node(node, label=str(node), title=str(node))

    for edge in G.edges:
        net.add_edge(edge[0], edge[1])

    for node in roots:
        net.get_node(node)['color'] = 'red'
            

    # Generate network with specific layout
    net.from_nx(G)
    net.show('graph_viz_sccs.html')

def get_target_nodes(root, G):
    target_nodes = [node for node, attributes in G.nodes(data=True) if attributes['cat'] == '1']
    return target_nodes





In [36]:
directory = '/home/cyril/ssh-rlkex/Generated_Graphs/output/basic/V_6_8_P1/24'
G = load_random_graph(directory)

#compute all sccs
sccs = list(nx.strongly_connected_components(G))



Loading graph from /home/cyril/ssh-rlkex/Generated_Graphs/output/basic/V_6_8_P1/24/27317-1643890740.graphml


In [37]:
#Detect if there are any cycles
if len(sccs) == len(G.nodes):
    print("No cycles detected")
else:
    print("Cycles detected")

Cycles detected


In [38]:
#Compare the number of nodes in the sccs with the number of nodes in the graph
scc_nodes = [node for scc in sccs for node in scc]
print(f'Number of nodes in the graph: {len(G.nodes)}')
print(f'Number of nodes in the sccs: {len(scc_nodes)}')
print(f'Number of SCCs: {len(sccs)}')

Number of nodes in the graph: 2816
Number of nodes in the sccs: 2816
Number of SCCs: 2705


In [39]:
#For each scc, check if it contains a node with no incoming edges to the scc
#this node is the root node of the scc

def get_root_nodes_from_scc(sccs, G):
    """If we consider each scc as a node, then we want to get the sccs that have no incoming edges."""
    root_nodes = []
    for scc in sccs:
        for node in scc:
            if len([n for n in G.predecessors(node) if n not in scc]) == 0:
                root_nodes.append(scc)
                break
    return root_nodes

root_nodes = get_root_nodes_from_scc(sccs, G)


Root nodes: ['n1338', 'n1340', 'n1341', 'n1344', 'n1347', 'n1349', 'n1351', 'n1354', 'n1356', 'n1359', 'n1362', 'n1369', 'n1372', 'n1375', 'n1377', 'n1380', 'n1382', 'n1384', 'n1387', 'n1391', 'n1395', 'n1403', 'n1406', 'n1409', 'n1411', 'n1413', 'n1925', 'n1927', 'n1928', 'n1930', 'n1933', 'n1936', 'n1987', 'n2008', 'n2058', 'n2065', 'n2071', 'n2081', 'n2111', 'n2113', 'n2116', 'n2117', 'n2118', 'n2120', 'n2121', 'n2175', 'n2176', 'n2177', 'n2178', 'n2179', 'n2180', 'n2181', 'n2182', 'n2258', 'n2259', 'n2266', 'n2495', 'n2496', 'n2497', 'n2498']
Number of root nodes: 60


In [40]:
plot_graph(G, sccs)

graph_viz_sccs.html


In [ ]:
import json
G = load_random_graph(directory)
nb_nodes_before = len(G.nodes)
# take all root nodes and the targets, and only keep the nodes that are on the path between them
root_nodes = get_root_nodes(G)
target_nodes = get_target_nodes(root_nodes, G)
print('Root nodes: ', root_nodes)
print('Target nodes: ', target_nodes)
nodes_to_keep = []
for root in root_nodes:
    for target in target_nodes:
        print(f'Computing shortest path between {root} and {target}')
        path = nx.shortest_path(G, root, target)
        nodes_to_keep.extend(path)
G = G.subgraph(nodes_to_keep)

#print number of kept nodes over total number of nodes
print('Kept nodes: ', len(nodes_to_keep))
print('Total nodes: ', nb_nodes_before)


# Convert the graph to a format that Cytoscape.js can understand
cytoscape_json = nx.cytoscape_data(G)['elements']

# Save the graph data to a JSON file
with open('graph_data.json', 'w') as f:
    json.dump(cytoscape_json, f)